In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 200

In [ ]:
def gaussdx(sigma, size):
    if size > 2 * 3 * sigma + 1:
        size = 2 * 3 * sigma + 1
    gaus = np.arange(-size,size+1)
    gaus = gaus.astype(float)
    gaus[:] = np.multiply( -1/(np.sqrt(2 * np.pi) * np.power(sigma,3)), np.multiply(gaus[:],np.exp(np.divide(-np.square(gaus[:]),2*np.square(sigma)))) )
    return gaus / sum(abs(gaus))

def gauss(sigma, size):
    if size > 3 * sigma + 1:
        size = 3 * sigma + 1
    gaus = np.arange(-size,size+1)
    gaus = gaus.astype(float)
    gaus[:] = np.multiply(1/(np.sqrt(2 * np.pi) * sigma), np.exp(np.divide(-np.square(gaus[:]),2*np.square(sigma))) )
    return gaus / sum(gaus)

def partial_derivatives(image, sigma, size):
    image = image.astype(float)
    g = gauss(size,sigma)
    d = gaussdx(size,sigma)
    ix = cv2.filter2D(cv2.filter2D(image,-1, g ,borderType=4),-1,cv2.transpose(np.flip(d)),borderType=4) 
    iy = cv2.filter2D(cv2.filter2D(image,-1, np.flip(d) ,borderType=4),-1,cv2.transpose(g),borderType=4)
    ixx = cv2.filter2D(cv2.filter2D(ix,-1, g ,borderType=4),-1,cv2.transpose(np.flip(d)),borderType=4) 
    ixy = cv2.filter2D(cv2.filter2D(ix,-1, np.flip(d) ,borderType=4),-1,cv2.transpose(g),borderType=4) 
    iyy = cv2.filter2D(cv2.filter2D(iy,-1, np.flip(d) ,borderType=4),-1,cv2.transpose(g),borderType=4) 
    return [ix,iy,ixx,ixy,iyy]

def hessian_points(image,sigma,size,thresh):
    _,_,xx,xy,yy = partial_derivatives(image, sigma, size)
    det = np.copy(xx)
    det[:] = np.power(sigma,4) * (xx[:] * yy[:] - np.square(xy[:]))
    thresholded = np.copy(det)

    thresholded = maxima_threshold(thresholded,thresh)
    print(np.amax(det))
    
    return [det,thresholded]

def maxima_threshold(c, thresh):
    
    det = np.pad(c, [(1,1), (1,1)], "constant", constant_values=0)
    det[ det < thresh ] = 0
    new = np.copy(det)
    indexes = np.nonzero(det)

    for i in range( len(indexes[0]) ):
        x = indexes[0][i]
        y = indexes[1][i]
        if det[x,y] < np.amax( det[x-1:x+2,y-1:y+2] ):
            new[x,y] = 0
        
    det = new
    det = det[1:len(det)-1,1:len(det[0])-1]
    return det

def draw_points(arr):
    plt.plot( np.nonzero(arr)[1], np.nonzero(arr)[0], 'x',markerfacecolor='none',color="#00FF55")
    # plt.plot( np.nonzero(arr)[1], np.nonzero(arr)[0], "ro")


i = cv2.imread("./data/test_points.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

rows = 2
cols = 3

a,b = hessian_points(i, 3, 9, 25000)
c,d = hessian_points(i, 6, 18, 50000)
e,f = hessian_points(i, 9, 27, 400000)

plt.subplot(rows,cols,1)
plt.imshow(a, cmap="gray")
plt.subplot(rows,cols,2)
plt.imshow(c, cmap="gray")
plt.subplot(rows,cols,3)
plt.imshow(e, cmap="gray")

plt.subplot(rows,cols,4)
plt.imshow(i, cmap="gray")
draw_points(b)

plt.subplot(rows,cols,5)
plt.imshow(i, cmap="gray")
draw_points(d)

plt.subplot(rows,cols,6)
plt.imshow(i, cmap="gray")
draw_points(f)

plt.show()



In [ ]:
def harris_points(image, sigma, threshold):
    x,y,_,_,_ = partial_derivatives(image, sigma, 3*sigma)
    k = gauss(1.6 * sigma, 3 * 1.6 * sigma)

    xx = sigma*sigma*cv2.filter2D(cv2.filter2D(np.multiply(x,x),-1,cv2.transpose(k)),-1,k)
    yy = sigma*sigma*cv2.filter2D(cv2.filter2D(np.multiply(y,y),-1,cv2.transpose(k)),-1,k)
    xy = sigma*sigma*cv2.filter2D(cv2.filter2D(np.multiply(x,y),-1,cv2.transpose(k)),-1,k)

    det = np.subtract(np.multiply(xx,yy), np.square(xy))
    trace = np.add(xx,yy).astype(np.float64)

    c = np.copy(det).astype(np.float64)
    c = np.subtract( det, 0.06*np.square(trace) )
    thresh = maxima_threshold(np.copy(c),threshold)
    print(np.amax(c))
    
    return [c, thresh]


i = cv2.imread("./data/test_points.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

rows = 2
cols = 3

a,b = harris_points(i, 3, 8111913)
c,d = harris_points(i, 6, 200000000)
e,f = harris_points(i, 9, 510205160)

plt.subplot(rows,cols,1)
plt.imshow(a, cmap="gray")
plt.subplot(rows,cols,2)
plt.imshow(c, cmap="gray")
plt.subplot(rows,cols,3)
plt.imshow(e, cmap="gray")

plt.subplot(rows,cols,4)
plt.imshow(i, cmap="gray")
draw_points(b)

plt.subplot(rows,cols,5)
plt.imshow(i, cmap="gray")
draw_points(d)

plt.subplot(rows,cols,6)
plt.imshow(i, cmap="gray")
draw_points(f)

plt.show()

In [ ]:
def simple_descriptors(I, pts, bins=16, rad=40, w=11):
	g = gauss(w, 3*w)
	d = gaussdx(w, 3*w)

	Ix = cv2.filter2D(I, cv2.CV_32F, g.T)
	Ix = cv2.filter2D(Ix, cv2.CV_32F, d)

	Iy = cv2.filter2D(I, cv2.CV_32F, g)
	Iy = cv2.filter2D(Iy, cv2.CV_32F, d.T)

	Ixx = cv2.filter2D(Ix, cv2.CV_32F, g.T)
	Ixx = cv2.filter2D(Ixx, cv2.CV_32F, d)

	Iyy = cv2.filter2D(Iy, cv2.CV_32F, g)
	Iyy = cv2.filter2D(Iyy, cv2.CV_32F, d.T)

	mag = np.sqrt(Ix**2+Iy**2)
	mag = np.floor(mag*((bins-1)/np.max(mag)))

	feat = Ixx+Iyy
	feat += abs(np.min(feat))
	feat = np.floor(feat*((bins-1)/np.max(feat)))

	desc = []

	for x,y in pts:
		minx = max(x-rad, 0)
		maxx = min(x+rad, I.shape[0])
		miny = max(y-rad, 0)
		maxy = min(y+rad, I.shape[1])
		r1 = mag[minx:maxx, miny:maxy].reshape(-1)
		r2 = feat[minx:maxx, miny:maxy].reshape(-1)
		
		a = np.zeros((bins, bins))
		for m, l in zip(r1, r2):
			a[int(m), int(l)] += 1

		a = a.reshape(-1)
		a/=np.sum(a)

		desc.append(a)

	return np.array(desc)

def find_correspondences(desc1, desc2):
	hellingers = np.zeros((len(desc1), len(desc2)))
	for i,x in enumerate(desc1):
		for j,y in enumerate(desc2):
			hellingers[i,j] = np.sqrt(np.multiply(0.5, np.sum(np.square(np.subtract(np.sqrt(x), np.sqrt(y))))))


	best_indexes = np.zeros(len(desc1))

	# for each point in desc1, get distances to points in desc2
	for i,x in enumerate(hellingers):
		best_indexes[i] = np.nonzero( (x == np.amin(x)) )[0][0]
	return best_indexes

i = cv2.imread("data/graf/graf1_small.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

i2 = cv2.imread("data/graf/graf2_small.jpg")
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)
i2 = cv2.cvtColor(i2, cv2.COLOR_RGB2GRAY)

_,t1 = hessian_points(i,3,9,30000)
points1 = np.nonzero(t1)
points1 = np.vstack((points1[0],points1[1])).T
d1 = simple_descriptors(i, points1)

_,t2 = hessian_points(i2,3,9,30000)
points2 = np.nonzero(t2)
points2 = np.vstack((points2[0],points2[1])).T
d2 = simple_descriptors(i2, points2)



In [ ]:
def display_matches(im1, im2, pts1, pts2, matches):
	first = 1
	second = 2
	if len(im1) != len(im2):
		length = max(len(im1),len(im2))
		first = np.zeros((length,len(im1[0])))
		second = np.zeros((length,len(im2[0])))
		first[:len(im1),:len(im1[0])] = im1
		second[:len(im2),:len(im2[0])] = im2
	else:
		first = im1
		second = im2

	I = np.hstack((first,second))
	w = first.shape[1]
	plt.clf()
	plt.imshow(I, cmap="gray")

	for i, j in matches:
		p1 = pts1[int(i)]
		p2 = pts2[int(j)]
		plt.plot(p1[1], p1[0], 'o',markerfacecolor='none',color="#00FF55")
		plt.plot(p2[1]+w, p2[0], 'o',markerfacecolor='none',color="#00FF55")
		plt.plot([p1[1], p2[1]+w], [p1[0], p2[0]], color="#00FF55")

	plt.draw()

korespondence = find_correspondences(d1,d2)
seq = np.arange(len(korespondence))
matches = (np.vstack((seq,korespondence)).T).astype(int)

display_matches(i,i2,points1,points2,matches)


In [ ]:
def find_matches(image1, image2):
    # _,t1 = harris_points(image1,3,200000)
    _,t1 = hessian_points(image1,3,9,30000)

    points1 = np.nonzero(t1)
    points1 = np.vstack((points1[0],points1[1])).T
    d1 = simple_descriptors(image1, points1)

    # _,t2 = harris_points(image2,3,200000)
    _,t2 = hessian_points(image2,3,9,30000)

    points2 = np.nonzero(t2)
    points2 = np.vstack((points2[0],points2[1])).T
    d2 = simple_descriptors(image2, points2)

    korespondence1 = find_correspondences(d1,d2)
    korespondence2 = find_correspondences(d2,d1)

    for i,k in enumerate(korespondence1):
        if not korespondence2[int(k)] == i:
            korespondence1[i] = -1

    matches = np.nonzero( (korespondence1 > -1) )[0]
    fromother = np.take(korespondence1, matches) 
    res = (np.vstack((matches,fromother)).T).astype(int)
    return [points1, points2, res]

i = cv2.imread("data/graf/graf1_small.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

i2 = cv2.imread("data/graf/graf2_small.jpg")
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)
i2 = cv2.cvtColor(i2, cv2.COLOR_RGB2GRAY)

p1,p2,mtch = find_matches(i,i2)
display_matches(i,i2,p1,p2,mtch)
plt.show()

In [ ]:
def estimate_homography(points):
    a = np.zeros(( len(points)*2, 9 ))
    a[::2,2] = 1
    a[1::2,5] = 1

    for i,point in enumerate(points):
        a[i*2,:2] = point[:2]
        a[i*2,6:] = [-point[0]*point[2], -point[1]*point[2], -point[2]]
        
        a[i*2+1,3:5] = point[:2]
        a[i*2+1,6:] = [-point[0]*point[3], -point[1]*point[3], -point[3]]
    
    _,_,a = np.linalg.svd(a)

    a = a[8,:]
    a = a / a[8]
    a = a.reshape((3,3))
    return a

def read_data(filename):
	with open(filename) as f:
		s = f.read()
	return np.fromstring(s, sep=' ')

newyork = read_data("data/newyork/newyork.txt").reshape(4,4)
p1 = newyork[:,:2].astype(int)
p2 = newyork[:,2:].astype(int)
matches = [[0,0],[1,1],[2,2],[3,3]]

i = cv2.imread("data/newyork/newyork1.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

i2 = cv2.imread("data/newyork/newyork2.jpg")
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)
i2 = cv2.cvtColor(i2, cv2.COLOR_RGB2GRAY)
h = estimate_homography(newyork)

display_matches(i,i2,p1,p2,matches)
plt.show()

i = cv2.warpPerspective(i,h,(250,250))
plt.imshow(i,cmap="gray")
plt.show()


graf = read_data("data/graf/graf.txt").reshape(4,4)
p1 = graf[:,:2].astype(int)
p2 = graf[:,2:].astype(int)
matches = [[0,0],[1,1],[2,2],[3,3]]

i = cv2.imread("data/graf/graf1.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

i2 = cv2.imread("data/graf/graf2.jpg")
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)
i2 = cv2.cvtColor(i2, cv2.COLOR_RGB2GRAY)
h = estimate_homography(graf)

display_matches(i,i2,p1,p2,matches)
plt.show()

i = cv2.warpPerspective(i,h,(800,640))
plt.imshow(i,cmap="gray")
plt.show()

In [ ]:
def find_correspondences2(desc1, desc2):
    hellingers = np.zeros((len(desc1), len(desc2)))
    for i,x in enumerate(desc1):
        for j,y in enumerate(desc2):
            hellingers[i,j] = np.sqrt(np.multiply(0.5, np.sum(np.square(np.subtract(np.sqrt(x), np.sqrt(y))))))

    best_indexes = np.zeros((len(desc1), 2))

    for i,x in enumerate(hellingers):
        best_indexes[i,:] = [np.nonzero( (x == np.amin(x)) )[0][0], np.amin(x)]

    return best_indexes

def find_matches_2(image1, image2):
    
    # tr1 = 3000000
    # tr2 = 2750000

    tr1 = 40000
    tr2 = 42000

    _,t1 = hessian_points(image1,3,9,tr1)
    # _,t1 = harris_points(image1,5,100000000)
    # plt.imshow(image1,cmap="gray")
    # draw_points(t1)
    # plt.show()
    points1 = np.nonzero(t1)
    points1 = np.vstack((points1[0],points1[1])).T
    d1 = simple_descriptors(image1, points1,8,8,2.9)

    _,t2 = hessian_points(image2,3,9,tr2)
    # _,t2 = harris_points(image2,5,100000000)
    # plt.imshow(image2,cmap="gray")
    # draw_points(t2)
    # plt.show()
    points2 = np.nonzero(t2)
    points2 = np.vstack((points2[0],points2[1])).T
    d2 = simple_descriptors(image2, points2,8,8,2.9)
 
    k1 = find_correspondences2(d2,d1)
    k2 = find_correspondences2(d1,d2)

    for i,k in enumerate(k1):
        if not k2[int(k[0]),0] == i:
            k1[i,0] = -1
    matchesn = np.nonzero( (k1[:,0] > -1) )[0]
    fromothern = np.take(k1, matchesn, axis=0)
    actual = np.zeros((len(matchesn),3))
    for g in range(len(matchesn)):
        actual[g,:2] = fromothern[g,:]
        actual[g,2] = matchesn[g]
    # actual = actual[actual[:, 1].argsort()]
    if len(actual) > 10:
        actual = actual[:10]  
    resn = (np.vstack((actual[:,0],actual[:,2])).T).astype(int)
    
    return [points1, points2, resn]

i = cv2.imread("data/newyork/newyork1.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

i2 = cv2.imread("data/newyork/newyork2.jpg")
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)
i2 = cv2.cvtColor(i2, cv2.COLOR_RGB2GRAY)

p1,p2,matches = find_matches_2(i,i2)

pnts = np.zeros( (len(matches), 4) )
for j,match in enumerate(matches):
    pnts[j,:2] = p2[match[1]]
    pnts[j,2:] = p1[match[0]]
    
h = estimate_homography(pnts)

display_matches(i,i2,p1,p2,matches)
plt.show()
i = cv2.warpPerspective(i,h,(250,250))
plt.imshow(i,cmap="gray")
plt.show()